In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


chef_template = ChatPromptTemplate.from_messages([
  ("system", "you are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
  ("human", "I want to cook {cuisine} food"),
])

chef_chain = chef_template | chat


In [54]:
veg_chef_template = ChatPromptTemplate.from_messages([
  ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't redically modify the recipe. If there is no alternative for a food, just say you don't know hot to replace it"),
  ("human","{recipe}"),
])

vege_chain = veg_chef_template | chat

final_chain = {"recipe" : chef_chain} | vege_chain

final_chain.invoke({
  "cuisine" : "Japanese"
})

Great choice! Japanese cuisine is known for its fresh and delicate flavors. Let's start with a simple and delicious recipe for Chicken Teriyaki.

Chicken Teriyaki Recipe:

Ingredients:
- 2 boneless, skinless chicken breasts
- 1/4 cup soy sauce
- 1/4 cup mirin (Japanese sweet rice wine)
- 2 tablespoons sugar
- 1 tablespoon vegetable oil
- 1 teaspoon grated ginger
- 1 clove garlic, minced
- Sesame seeds and green onions for garnish

Instructions:
1. Slice the chicken breasts into thin strips or bite-sized pieces.
2. In a small bowl, mix together soy sauce, mirin, sugar, ginger, and garlic to make the teriyaki sauce.
3. Heat vegetable oil in a large skillet over medium-high heat.
4. Add the chicken to the skillet and cook until browned on all sides.
5. Pour the teriyaki sauce over the chicken and stir to coat.
6. Reduce heat to low and simmer for about 5-7 minutes, or until the sauce has thickened and the chicken is cooked through.
7. Serve the Chicken Teriyaki over steamed rice and garni

AIMessageChunk(content="For a vegetarian twist on the classic Chicken Teriyaki recipe, you can easily substitute the chicken with tofu or seitan. Here's how you can prepare the alternative ingredients:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe.\n   - Press the tofu to remove excess water: Place the block of tofu between paper towels or kitchen towels, then place a heavy object on top (like a plate with a can on it) for about 15-30 minutes.\n   - Cut the tofu into thin strips or bite-sized pieces similar to the chicken.\n   - You can either pan-fry the tofu in a little oil until golden brown before adding the teriyaki sauce or simply add the tofu directly to the sauce to simmer.\n\n2. **Seitan**:\n   - Seitan is a high-protein meat substitute made from gluten, the protein in wheat.\n   - You can either make your own seitan from vital wheat gluten or purchase pre-made seitan from the store.\n   - Cut the seitan into thin strips or bite-sized pieces similar to the